**Ensemble Tekniklerinden olan Stacking Ensemble Modeli kuracağız.**

Burda sadece Stacking Tekniğini uygulamaya çalıştığım için herhangi bir görselleştirme yapmadım.

**Veri:** iris

In [77]:
import numpy as np
import seaborn as sns
import pandas as pd

data=sns.load_dataset("iris")
df=data.copy()
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal_length    150 non-null float64
sepal_width     150 non-null float64
petal_length    150 non-null float64
petal_width     150 non-null float64
species         150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [79]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
sepal_length,150.0,5.843333,0.828066,4.3,5.1,5.80,6.4,7.9
sepal_width,150.0,3.057333,0.435866,2.0,2.8,3.00,3.3,4.4
petal_length,150.0,3.758000,1.765298,1.0,1.6,4.35,5.1,6.9
petal_width,150.0,1.199333,0.762238,0.1,0.3,1.30,1.8,2.5


In [0]:
y=df["species"]
X=df.drop("species",axis=1)

In [5]:
df.species.unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

**Modeller için species özniteliğini sayısal veriye çeviriyoruz.**

In [6]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder().fit(y)
y=le.transform(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

**Train ve Test Verisinin Ayrımı**

Veriyi 2'ye ayırıyoruz. İlk kısmını 3 modeli eğitmek için kullanıyoruz. 2. kısmı 3 modele verip tahmin ettiriyoruz. Ve bu çıkan tahminlerle de Blend yapacak modeli eğitiyoruz.

x_train_model :  

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#X_test ve y_test çıktı alacağımız son modelde test için kullanılacak.
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=31)

#Burda Modelleri eğitmek ve tahmin yaptırmak için X_train'i 2 ye böldük.
X_train_model, X_test_model=X_train.iloc[:len(X_train)//2,:], X_train.iloc[len(X_train)//2:,:]
y_train_model, y_test_model=y_train[:len(y_train)//2], y_train[len(y_train)//2:]

**Naive Bayes Modeli**

In [52]:
from sklearn.naive_bayes import GaussianNB
nb=GaussianNB().fit(X_train_model,y_train_model)
nb

GaussianNB(priors=None, var_smoothing=1e-09)

**RBF SVC Modeli**

In [53]:
from sklearn.svm import SVC
svc=SVC(kernel="linear").fit(X_train_model,y_train_model)
svc

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

**CART - DecisionTree Modeli**

In [55]:
from sklearn.tree import DecisionTreeClassifier
cart=DecisionTreeClassifier().fit(X_train_model,y_train_model)
cart

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

**Stacking Modelini Kurma**

Burda 3 Modelin Train kümesinde tahmin ettiği değerlerini giriş alıp çıkış olarak Tür tahmini yapan Yapay Sinir Ağı modeli kuruyoruz.

In [0]:
def reshape(x):
  return np.reshape(x,(x.shape[0],1))

In [0]:
def modeller(test):
  nb_pred=reshape(nb.predict(test))
  svc_pred=reshape(svc.predict(test))
  cart_pred=reshape(cart.predict(test))
  return np.concatenate((nb_pred,svc_pred,cart_pred),axis=1)

In [87]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

#X_traini ayırdığımız 2. parçasıyla modellere tahmin yaptırıyoruz.
scaler=StandardScaler().fit(modeller(X_test_model))
#X_train_scaled=scaler.transform(modeller(X_test_model))

#Modellerin çıkardığı tahminle Yapay Sinir Ağını eğitiyoruz.
mlp=MLPClassifier().fit(X_train_scaled,y_test_model)
mlp

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

**Tahmin**

In [88]:
X_test_scaled=scaler.transform(modeller(X_test))
accuracy_score(y_test,mlp.predict(X_test_scaled))

0.9333333333333333

**Böylelikle Ensemble Tekniklerinden olan Stacking'i daha iyi anlamaya çalıştık.**